# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [107]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [108]:
df.columns = df.columns.str.lower()
df.head()

,unnamed: 0,customer,state,customer lifetime value,response,coverage,education,effective to date,employmentstatus,gender,...,number of open complaints,number of policies,policy type,policy,renew offer type,sales channel,total claim amount,vehicle class,vehicle size,vehicle type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [109]:
df.columns= df.columns.str.replace(" ", "_")
print(df.columns)

Index(['unnamed:_0', 'customer', 'state', 'customer_lifetime_value',
       'response', 'coverage', 'education', 'effective_to_date',
       'employmentstatus', 'gender', 'income', 'location_code',
       'marital_status', 'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')


In [110]:
df.rename(columns= {'unnamed:_0': 'columna_1'}, inplace=True)
print(df.columns)

Index(['columna_1', 'customer', 'state', 'customer_lifetime_value', 'response',
       'coverage', 'education', 'effective_to_date', 'employmentstatus',
       'gender', 'income', 'location_code', 'marital_status',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')


In [111]:
new_df= df[["customer", "total_claim_amount", "response"]]
new_df.head()

,customer,total_claim_amount,response
0,DK49336,292.800000,No
1,KX64629,744.924331,No
2,LZ68649,480.000000,No
3,XL78013,484.013411,Yes
4,QA50777,707.925645,No


In [112]:
condition = (new_df["total_claim_amount"]> 1000) & (new_df["response"] == "Yes")
condition
filtered_df = new_df[condition]
filtered_df

,customer,total_claim_amount,response
189,OK31456,1358.400000,Yes
236,YJ16163,1358.400000,Yes
419,GW43195,1027.200000,Yes
442,IP94270,1261.319869,Yes
587,FJ28407,1027.000029,Yes
654,VV47792,1027.200000,Yes
680,QL45827,1027.200000,Yes
815,MA15172,1300.800000,Yes
1276,WA45944,1027.200000,Yes
1390,FJ28407,1027.000029,Yes


In [113]:
filtered_df.reset_index()

,index,customer,total_claim_amount,response
0,189,OK31456,1358.400000,Yes
1,236,YJ16163,1358.400000,Yes
2,419,GW43195,1027.200000,Yes
3,442,IP94270,1261.319869,Yes
4,587,FJ28407,1027.000029,Yes
5,654,VV47792,1027.200000,Yes
6,680,QL45827,1027.200000,Yes
7,815,MA15172,1300.800000,Yes
8,1276,WA45944,1027.200000,Yes
9,1390,FJ28407,1027.000029,Yes


In [114]:
df[(df.response == "Yes")]
pivot_df = df.pivot_table(index='gender', columns='policy_type', values=['total_claim_amount'], aggfunc='mean').round(2)
pivot_df

total_claim_amount                           
policy_type     Corporate Auto Personal Auto Special Auto
gender                                                   
F                       397.80        413.24       458.14
M                       462.22        459.92       420.36

In [115]:
new_pivot_df = df.pivot_table(index='state', columns='number_of_policies', values=['customer'], aggfunc='size').round(2)
new_pivot_df

number_of_policies,1,2,3,4,5,6,7,8,9
state,,,,,,,,,
Arizona,671,483,231,88,107,82,95,78,102
California,1234,889,475,172,166,131,173,158,154
Nevada,382,259,126,33,23,54,44,34,38
Oregon,1016,748,372,130,130,110,144,131,128
Washington,346,217,97,37,37,28,44,35,47


In [116]:
customer_state[(df["state"].value_counts() > 500)]

California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: state, dtype: int64

In [117]:
pi_df = df.pivot_table(index='gender', columns='education', values=['customer_lifetime_value'], aggfunc='max').round(2)
pi_df

customer_lifetime_value                                           \
education                Bachelor   College    Doctor High School or Below   
gender                                                                       
F                        73225.96  61850.19  44856.11             55277.45   
M                        67907.27  61134.68  32677.34             83325.38   

                     
education    Master  
gender               
F          51016.07  
M          50568.26

In [118]:
pi_df = df.pivot_table(index='gender', columns='education', values=['customer_lifetime_value'], aggfunc='min').round(2)
pi_df

customer_lifetime_value                                         \
education                Bachelor  College   Doctor High School or Below   
gender                                                                     
F                         1904.00  1898.68  2395.57              2144.92   
M                         1898.01  1918.12  2267.60              1940.98   

                    
education   Master  
gender              
F          2417.78  
M          2272.31

In [119]:
pi_df = df.pivot_table(index='gender', columns='education', values=['customer_lifetime_value'], aggfunc='mean').round(2)
pi_df

customer_lifetime_value                                         \
education                Bachelor  College   Doctor High School or Below   
gender                                                                     
F                         7874.27  7748.82  7328.51              8675.22   
M                         7703.60  8052.46  7415.33              8149.69   

                    
education   Master  
gender              
F          8157.05  
M          8168.83

In [121]:
pi_df = df.pivot_table(index= ["education", "gender"], values=['customer_lifetime_value'], aggfunc={"customer_lifetime_value": ["max", "min", "mean"]}).round(2)
pi_df

customer_lifetime_value                  
                                                max     mean      min
education            gender                                          
Bachelor             F                     73225.96  7874.27  1904.00
                     M                     67907.27  7703.60  1898.01
College              F                     61850.19  7748.82  1898.68
                     M                     61134.68  8052.46  1918.12
Doctor               F                     44856.11  7328.51  2395.57
                     M                     32677.34  7415.33  2267.60
High School or Below F                     55277.45  8675.22  2144.92
                     M                     83325.38  8149.69  1940.98
Master               F                     51016.07  8157.05  2417.78
                     M                     50568.26  8168.83  2272.31

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [122]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
df['month'] = df['effective_to_date'].dt.month

In [135]:
df_bonus = df.pivot_table(index=['state',"policy_type"], columns='month', values=['number_of_policies'], aggfunc='sum').round(2)
df_bonus

number_of_policies      
month                                      1     2
state      policy_type                            
Arizona    Corporate Auto                667   525
           Personal Auto                2251  2220
           Special Auto                  134   119
California Corporate Auto               1379  1164
           Personal Auto                4111  3594
           Special Auto                  183   171
Nevada     Corporate Auto                334   257
           Personal Auto                1093   991
           Special Auto                   66    30
Oregon     Corporate Auto                958   901
           Personal Auto                3532  2897
           Special Auto                  207   171
Washington Corporate Auto                265   276
           Personal Auto                1070   897
           Special Auto                   23    52

In [137]:
df_bonus_2 = df.pivot_table(index=['state'], columns='month', values=['number_of_policies'], aggfunc='sum').round(2)
df_bonus_2

number_of_policies      
month                       1     2
state                              
Arizona                  3052  2864
California               5673  4929
Nevada                   1493  1278
Oregon                   4697  3969
Washington               1358  1225

In [145]:
df_groupby = df.groupby(['state', 'month']).size().reset_index(name='number_of_policies')
df_groupby

,state,month,number_of_policies
0,Arizona,1,1008
1,Arizona,2,929
2,California,1,1918
3,California,2,1634
4,Nevada,1,551
5,Nevada,2,442
6,Oregon,1,1565
7,Oregon,2,1344
8,Washington,1,463
9,Washington,2,425


In [189]:
df_total_groupby = df_groupby.groupby('state')['number_of_policies'].sum().reset_index().sort_values(by="number_of_policies", ascending=False)
df_total_groupby

,state,number_of_policies
1,California,3552
3,Oregon,2909
0,Arizona,1937
2,Nevada,993
4,Washington,888


In [191]:
top_3_states = df_total_groupby.head(3)
top_3_states

,state,number_of_policies
1,California,3552
3,Oregon,2909
0,Arizona,1937


In [193]:
df_top_3_monthly = df_groupby[df_groupby['state'].isin(top_3_states['state'])]
df_top_3_monthly

,state,month,number_of_policies
2,California,1,1918
3,California,2,1634
6,Oregon,1,1565
7,Oregon,2,1344
0,Arizona,1,1008
1,Arizona,2,929


In [217]:
df_marketing = df.groupby('sales_channel')['response'].value_counts()
df_marketing

sales_channel  response
Agent          No          3148
               Yes          742
Branch         No          2539
               Yes          326
Call Center    No          1792
               Yes          221
Web            No          1334
               Yes          177
Name: response, dtype: int64

In [219]:
df_marketing.unstack(fill_value=0)["Yes"]

sales_channel
Agent          742
Branch         326
Call Center    221
Web            177
Name: Yes, dtype: int64